In [ ]:
#import library

import mne
import numpy as np
import os
import matplotlib.pyplot as plt
import statistics
import pyeeg
import librosa
from scipy import stats

In [ ]:
import pandas as pd
df = pd.read_csv('potongan_seizure.csv')

In [ ]:
path_data = "C:/Users/Asus/Downloads/Bisa AI/DATASET"
#ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)

In [ ]:
#function for preprocessing_data eeg

def preprocessing_data(file,channel,tawal,takhir):
    x=mne.io.read_raw_edf(file)
    x.crop(tmin=tawal, tmax=takhir)
    #ica.fit(x)
    x.load_data()
    #ica.apply(x)
    x.pick_types(x.info,eeg=True, include=[], exclude='bads', selection=channel)
    x.filter(1, 35., fir_design='firwin')
    averaged=sum(x.get_data())/len(x.ch_names)
    return averaged    

In [ ]:
signal=[]
channel=['FP1-F7','F7-T7','T7-P7','P7-O1','FP1-F3','F3-C3','C3-P3','P3-O1','FP2-F4',
            'F4-C4','C4-P4','P4-O2','FP2-F8','F8-T8','T8-P8-0','T8-P8','P8-O2','FZ-CZ','CZ-PZ']

#load raw eeg data (.edf)
for x in range(len(df)):
    file_name = os.path.join(path_data, df['nama file'][x])
    signal.append(preprocessing_data(file_name,channel,df['mulai (s)'][x],df['selesai (s)'][x]));

In [ ]:
crop=[]
jumlah=[]

#crop eeg data 5 sec each
for i in range(len(signal)):
    bagi=len(signal[i])//1280
    jumlah.append(bagi)
    for j in range(bagi):
        crop.append(signal[i][(1280*j):(1280*(j+1))+1])

In [ ]:
psd=[]

#calculate power spectral density using welch method
for n in range(len(crop)):
    welch, freq= mne.time_frequency.psd_array_welch(crop[n], sfreq=256, fmin=1, fmax=35, n_fft=1280,n_overlap=256/2,
                                                    n_per_seg=None, n_jobs=1, average='mean', verbose=None)
    log_psd= 10 * np.log10(welch)
    psd.append(log_psd)

In [ ]:
#function for low level feature extractions
def low_level(signal,welch):
    sig_mean=statistics.mean(signal)
    sig_var=statistics.variance(signal)
    sig_sd=statistics.stdev(signal)
    sig_kurt=stats.kurtosis(signal, axis=0, fisher=True, bias=True, nan_policy='propagate')
    sig_skew=stats.skew(signal, axis=0, bias=True)
    sig_centroid=librosa.feature.spectral_centroid(y=signal, sr=256)[0,0]
    bin_power=pyeeg.bin_power(signal, [0.5,4,7,12,30], 256) 
    sig_specent=pyeeg.spectral_entropy(signal, [0.5,4,7,12,30], 256, Power_Ratio=bin_power[1])
    
    psd_mean=statistics.mean(welch)
    psd_var=statistics.variance(welch)
    psd_sd=statistics.stdev(welch)
    psd_kurt=stats.kurtosis(welch, axis=0, fisher=True, bias=True, nan_policy='propagate')
    psd_skew=stats.skew(welch, axis=0, bias=True)
    rat_delta=bin_power[1][0]
    rat_theta=bin_power[1][1]
    rat_alpha=bin_power[1][2]
    rat_beta=bin_power[1][3]
    
    return sig_mean,sig_var,sig_sd,sig_kurt,sig_skew,sig_centroid,sig_specent,psd_mean,psd_var,psd_sd,psd_kurt,psd_skew,rat_delta,rat_theta,rat_alpha,rat_beta

In [ ]:
data_low_level = []
#low level feature extraction from eeg
for x in range(len(crop)):
    try:
        data_low_level.append(low_level(crop[x],psd[x]))
    except:
        print("Error Baca File")

In [ ]:
#convert the feature into dataframe
dataset = pd.DataFrame(data=data_low_level, columns=['sig_mean','sig_var','sig_sd','sig_kurt','sig_skew','sig_centroid',
                                                     'sig_specent','psd_mean','psd_var','psd_sd','psd_kurt','psd_skew',
                                                     'rat_delta','rat_theta','rat_alpha','rat_beta'])

In [ ]:
df_norm = pd.read_csv('potongan_normal.csv')

In [ ]:
signal_norm=[]

for x in range(len(df_norm)):
    file_name = os.path.join(path_data, df_norm['nama file'][x])
    signal_norm.append(preprocessing_data(file_name,channel,df_norm['mulai (s)'][x],df_norm['selesai (s)'][x]));

In [ ]:
crop_norm=[]
jumlah_norm=[]
for i in range(len(signal_norm)):
    bagi=len(signal_norm[i])//1280
    jumlah_norm.append(bagi)
    for j in range(bagi):
        crop_norm.append(signal_norm[i][(1280*j):(1280*(j+1))+1])

In [ ]:
psd_norm=[]
for n in range(len(crop_norm)):
    welch, freq= mne.time_frequency.psd_array_welch(crop_norm[n], sfreq=256, fmin=1, fmax=35, n_fft=1280,n_overlap=256/2,
                                                    n_per_seg=None, n_jobs=1, average='mean', verbose=None)
    log_psd= 10 * np.log10(welch)
    psd_norm.append(log_psd)

In [ ]:
data_low_level_norm = []
for x in range(len(crop_norm)):
    try:
        data_low_level_norm.append(low_level(crop_norm[x],psd_norm[x]))
    except:
        print("Error Baca File")

In [ ]:
dataset_2 = pd.DataFrame(data=data_low_level_norm, columns=['sig_mean','sig_var','sig_sd','sig_kurt','sig_skew','sig_centroid',
                                                     'sig_specent','psd_mean','psd_var','psd_sd','psd_kurt','psd_skew',
                                                     'rat_delta','rat_theta','rat_alpha','rat_beta'])

In [ ]:
dataset.describe()

In [ ]:
dataset_2.describe()

In [ ]:
import librosa.display

df_spec = pd.read_csv('potongan_spectrogram.csv')

signal_spec=[]

for x in range(len(df_spec)):
    file_name = os.path.join(path_data, df_spec['nama file'][x])
    signal_spec.append(preprocessing_data(file_name,channel,df_spec['mulai (s)'][x],df_spec['selesai (s)'][x]))

In [ ]:
#spectrogram
for i in range(len(signal_spec)):
    S = librosa.feature.melspectrogram(y=signal_spec[i], sr=256, n_mels=256/4, fmax=35)
    plt.figure(figsize=(5, 5)) 
    S_dB = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=256,fmax=35) 
    plt.axis('off')
    plt.tight_layout() 
    plt.savefig("norm_"+str(i).zfill(3)+".jpg", bbox_inches='tight')


for i in range(len(signal)):
    S = librosa.feature.melspectrogram(y=signal[i], sr=256, n_mels=256/4, fmax=35)
    plt.figure(figsize=(5, 5)) 
    S_dB = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=256,fmax=35) 
    plt.axis('off')
    plt.tight_layout() 
    plt.savefig("seizure_"+str(i).zfill(3)+".jpg", bbox_inches='tight')

In [ ]:
#save dataframe into csv
dataset.to_csv('data_seizure.csv', index = False)

In [ ]:
dataset_2.to_csv('data_normal.csv', index = False)